# Notebook 2: Collocation points for PINN

In this notebook, we will come back to the loss function we defined before for PINNs, introducing the concept of collocation points and using them to enhance the performance of the network from the first notebook.

